In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

from affine import Affine

from rasterio.control import GroundControlPoint
from rasterio import transform

import py_gamma as pg

/usr/local/GAMMA/GAMMA_SOFTWARE-20211208/py_gamma.py:594: SyntaxWarning: invalid escape sequence '\C'
  with OpenKey(HKEY_CURRENT_USER, "Software\Classes\http\shell\open\command") as key:


In [5]:
meta = pd.read_excel(Path('/home/rdcrlzh1s/radarsat/data/RS2_collection.xlsx'), parse_dates=['Acq. Date'])

In [7]:
meta[meta['AOI '] == 'Pituffik']

,AOI,Acq. Date,Start time,Orbit,Abs orb,Beam,Polarity,inc. angle,lat/long center,lat/long NW,lat/long NE,lat/long SW,lat/long SE
5,Pituffik,2023-06-15,21:52:11.470000,235-83A,80924.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
7,Pituffik,2023-06-22,21:48:00.520000,235-183A,81024.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W
10,Pituffik,2023-07-09,21:52:11.470000,236-83A,81267.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
12,Pituffik,2023-07-16,21:48:00.520000,236-183A,81367.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W
17,Pituffik,2023-08-02,21:52:11.470000,237-83A,81610.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
19,Pituffik,2023-08-09,21:48:00.520000,237-183A,81710.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W
25,Pituffik,2023-08-26,21:52:11.470000,238-83A,81953.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
27,Pituffik,2023-09-02,21:48:00.520000,238-183A,82053.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W
33,Pituffik,2023-09-19,21:52:11.470000,239-83A,82296.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
35,Pituffik,2023-09-26,21:48:00.520000,239-183A,82396.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W


In [2]:
import sys
from io import StringIO
class Capturing(list):
    """
    Captures print output of pygamma for parsing.
    """
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

In [3]:
def create_png(data_fp, par_fp = None, image_dir = None, width = None):
    if not par_fp and not width:
        par_fp = create_par_fp(data_fp)
        if not par_fp.exists(): raise ValueError()
    
    if not image_dir: image_dir = Path('./').resolve()

    if par_fp:
        arr= pg.read_image(data_fp, par = str(par_fp))
    elif width:
        arr= pg.read_image(data_fp, width = width) 
    
    print(arr)

    if np.iscomplexobj(arr):
        arr = np.angle(arr)

    vmin, vmax = np.quantile(arr, [0.01, 0.99])

    plt.imshow(arr, aspect = 'auto', vmax = vmin, vmin = vmax)

    plt.colorbar()

    plt.savefig(image_dir.joinpath(data_fp.stem).with_suffix(data_fp.suffix + '.png'))

    plt.close('all')

def create_par_fp(fp):
    return fp.with_suffix(fp.suffix + '.par')

f = Path('/home/rdcrlzh1s/radarsat/data/test/outputs/2023-10-13_2023-10-20.cor')
p = Path('/home/rdcrlzh1s/radarsat/data/test/outputs/2023-10-13_2023-10-20.diff_par')
i = Path('/home/rdcrlzh1s/radarsat/data/test/images')
create_png(f, p , image_dir = i)

image:
  width: 2272  number of lines: 1060
data read from image:
  column offset (x0): 0  row offset (y0): 0  
  width: 2272  number of lines: 1060
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
def get_width(par_fp: Path):
    """
    get width of gamma image from par file
    """
    par = pg.ParFile(str(par_fp))
    if 'range_samples' in par.par_keys:
        width = int(par.get_dict()['range_samples'][0])
    elif 'width' in par.par_keys:
        width = int(par.get_dict()['width'][0])
    elif 'range_samp_1' in par.par_keys:
        width = int(par.get_dict()['range_samp_1'][0])
    elif 'interferogram_width' in par.par_keys:
        width = int(par.get_dict()['interferogram_width'][0])
    return width

get_width('/home/rdcrlzh1s/radarsat/data/test/processing/2023-10-13_2023-10-20.diff_par')

/usr/local/GAMMA/GAMMA_SOFTWARE-20211208/py_gamma.py:594: SyntaxWarning: invalid escape sequence '\C'
  with OpenKey(HKEY_CURRENT_USER, "Software\Classes\http\shell\open\command") as key:


FileNotFoundError: [Errno 2] No such file or directory: '/home/rdcrlzh1s/radarsat/data/test/processing/2023-10-13_2023-10-20.diff_par'

In [6]:
arr= pg.read_image(f, par = str(p))
vmin, vmax = np.quantile(arr, [0.01, 0.99])

plt.imshow(arr, aspect = 'auto', vmax = vmin, vmin = vmax)

image:
  width: 2215  number of lines: 1080
data read from image:
  column offset (x0): 0  row offset (y0): 0  
  width: 2215  number of lines: 1080


In [8]:
plt.imshow([[0,0,0],[1,1,1],[2,2,2]])